# Imports

In [1]:
import pandas as pd
import re

import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import wrangle as w
import explore as e
import modeling as m

# Acquire

* Data aquired from [Kaggle](https://www.kaggle.com/satpreetmakhija/netflix-movies-and-tv-shows-2021) on 2/04/2022
* Each observation represents on movie or film series 
* Because the distinction between a single movie an a multi-part series is not relevant to this study I will be referring to each as a movie for the duration of this study
* The original data set had 5967 rows and 13 columns

In [2]:
# read the csv into pandas
df = w.get_movie_data()
df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


# Prepare

In [3]:
df_nope = w.prep_movie_data(w.get_movie_data())
df_nope.head()

/Users/johnsalas/Desktop/git_repositories/data_science_projects/guess_the_genre_netflix/wrangle.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].apply(lambda value: value.lower())
/Users/johnsalas/Desktop/git_repositories/data_science_projects/guess_the_genre_netflix/wrangle.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genres'] = df['Genres'].apply(lambda value: value.lower())


,description,genre
0,docuseries take deep dive lucrative wellness i...,reality tv
1,grisly virus rampage city lone man stay locked...,"horror movies, international movies, thrillers"
2,diary anne frank story retold alongside five h...,"documentaries, international movies"
3,kenya barris family navigate relationship race...,tv comedies
4,pawesome documentary explores feline friend be...,"documentaries, international movies"


In [ ]:
df = w.get_movie_data()
df.head()

In [ ]:
df = df[['Description','Genres']]
df.head()

In [ ]:
# Lowercase all of the letters in both columns
df['Description'] = df['Description'].apply(lambda value: value.lower())
df['Genres'] = df['Genres'].apply(lambda value: value.lower())
df.head()

In [ ]:
# rename columns
df.rename(columns={'Description':'description', 'Genres':'genre'}, inplace = True)
df.head()

In [ ]:
# remove non-ascii characters from description text 
df['description'] = df['description'].apply(lambda value: unicodedata.normalize('NFKD', value)\
                                                                     .encode('ascii', 'ignore')\
                                                                     .decode('utf-8', 'ignore'))
df.head()

In [ ]:
# remove special characters from description text
df['description'] = df['description'].apply(lambda value: re.sub(r"[^a-z0-9\s]", '', value))

df.head()

In [ ]:
# create tokenizer object
tokenizer = nltk.tokenize.ToktokTokenizer()

# tokenize text in description
df['description'] = df['description'].apply(lambda value: tokenizer.tokenize(value, return_str=True))

df.head()

In [ ]:


def lemmatizer(value):
    '''Takes in a value from a pandas column and returns the value lemmatized'''
    
    # create lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    
    # get list of lemmatized words in value
    value_lemmas = [wnl.lemmatize(word) for word in value.split()]
    
    # turn list or words back into a string and return value
    return ' '.join(value_lemmas)

In [ ]:
# lemmatize the text in description
df['description'] = df['description'].apply(lambda value: lemmatizer(value))

df.head()

In [ ]:
def remove_stopwords(value):
    
    # get list english language stopwords list from nlt
    stopword_list = stopwords.words('english')
    
    # split words in pandas value into a list and remove words from the list that are in stopwords
    value_words = value.split()
    filtered_list = [word for word in value_words if word not in stopword_list]
    
    # convert list back into string and return value
    return ' '.join(filtered_list)

In [ ]:
# remove stopwords from text in description
df['description'] = df['description'].apply(lambda value: remove_stopwords(value))

df.head()

In [ ]:
def remove_cinima_type(value):
    '''take in genre text from a pandas column
       remove text indicating standalone movie or series
       return remainder of the text'''
     
    value = value.replace(' tv','') 
    value = value.replace('tv ','')
    value = value.replace(' shows','')
    value = value.replace(' movies','')
    value = value.replace(' series','')
    value = value.replace(' features','')
    
    value = [genre.strip() for genre in value.split(',')]
    
    return ','.join(value)

In [ ]:
df['genre'] = df['genre'].apply(lambda value: remove_cinima_type(value))

df.head()

In [ ]:
df.info()

In [ ]:
master_list = []

for text in set(df.genre.to_list()):
    
    value_genres = text.split(', ')
    
    for item in value_genres:
        
        master_list.append(item)
        
print(set(master_list), len(set(master_list)))

In [ ]:
def get_genre_list(value):
    '''takes in string list of genes from a pandas column
       creates a python list of those genres 
       removes genres in the cut list 
       returns created list'''
    
    cut_list = ['international', 
                'teen', 
                'korean', 
                'anime', 
                'classic & cult', 
                "kids'", 
                'cult', 
                'movies', 
                'shows',
                'spanish-language', 
                'british', 
                'children & family', 
                'anime series',
                'lgbtq',
                'classic', 
                'international',
                'independent',
                'sci-fi & fantasy',
                'sports',
                'faith & spirituality',
                'stand-up comedy & talk',
                'mysteries']
    
    genre_list = value.split(',')
    genre_list = [genre.strip() for genre in genre_list]
    
    return [genre for genre in genre_list if genre not in cut_list]

In [ ]:
df['genre_list'] = df['genre'].apply(lambda value: get_genre_list(value))

df.head()

In [ ]:
# drop rows with empty lists looses 469 rows
print(df.shape)
df = df[df['genre_list'].map(lambda d: len(d)) > 0]
print(df.shape)

In [ ]:
def merge_genres(value, merge_list, replacement):
    '''Take in a pandas value that is a list of genres
       a merge list and a replacement string
       If one of the genres in genre_list is in merge_list 
       return a list containing the replacement string
       otherwise return original list'''

    # builds list of genres from value matching merge_list
    check_list = [genre for genre in value if genre in merge_list]

    # if check_list is not empty return list with just documentary
    if len(check_list) > 0:
        
        return [replacement]
    
    # otherwise return original list
    else:
        
        return value

In [ ]:
merge_list = ['docuseries', 'documentary', 'documentaries']
df['genre_list'] = df['genre_list'].apply(lambda value: merge_genres(value,merge_list,'documentaries'))

df.head()

In [ ]:
merge_list = ['reality']
df['genre_list'] = df['genre_list'].apply(lambda value: merge_genres(value,merge_list,'reality'))

df.genre_list.value_counts().head(30)

In [ ]:
merge_list = ['music & musicals']
df['genre_list'] = df['genre_list'].apply(lambda value: merge_genres(value,merge_list,'music & musicals'))

df.genre_list.value_counts().head(30)

In [ ]:
merge_list = ['crime']
df['genre_list'] = df['genre_list'].apply(lambda value: merge_genres(value,merge_list,'crime'))

df.genre_list.value_counts().head(30)

In [ ]:
def fuse_genre(value, fuse_list, replacement):
    
    # builds list of genres from value matching fuse_list
    check_list = [genre for genre in value if genre in fuse_list]

    # if the length of check_list is equal to the length of value return replacement 
    if (len(check_list) == len(value)) and (len(value) == 2):
        
        return [replacement]
    
    # otherwise return original list
    else:
        
        return value

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['romantic', 'comedies'], 'romantic comedies'))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['comedies', 'dramas'], 'dramatic comedies'))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['dramas', 'romantic'], 'romantic'))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['dramas', 'thrillers']  , 'thrillers'))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['action & adventure', 'dramas']  , 'thrillers'))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['dramas', 'horror']  , 'horror'))

df.genre_list.value_counts().head(30)

In [ ]:
def remove_genre(value, genre, val_len):
    
    if (len(value) >= val_len) and (genre in value):
        
        value.remove(genre)
        
    return value

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: remove_genre(value,'dramas', 3))

df.genre_list.value_counts().head(30)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['romantic', 'comedies'], 'romantic comedies'))

df.genre_list.value_counts().head(30)

In [ ]:
merge_list = ['horror']
df['genre_list'] = df['genre_list'].apply(lambda value: merge_genres(value,merge_list,'horror'))

df.genre_list.value_counts().head(99)

In [ ]:
df['genre_list'] = df['genre_list'].apply(lambda value: fuse_genre(value, ['action & adventure', 'comedies'], 'action & adventure comedies'))

df.genre_list.value_counts().head(99)

In [ ]:
print(df.shape)
df = df[df['genre_list'].map(lambda d: len(d)) == 1]
print(df.shape)

In [ ]:
df.head()

In [ ]:
df['genre'] = df.genre_list.apply(lambda value: value[0])
df.head()

In [ ]:
df = df.drop(columns = 'genre_list')

df.head()

In [ ]:
set(df.genre.to_list())